## 汽油辛烷值优化建模

以下解题过程中：
+ 参数定义
    - 《附件一：325个样本数据.xlsx》以符号 $\displaystyle{\mathcal{X}}_{s}$ 表示
    - 《附件三：285号和313号样本原始数据.xlsx》以符号 $\displaystyle{\mathcal{X}}_{r}$ 表示

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
sample_data = "../附件一：325个样本数据.xlsx"
raw_data = "../附件三：285号和313号样本原始数据.xlsx"
adjust_data = "../附件四：354个操作变量信息.xlsx"

In [ ]:
raws = pd.read_excel(io=raw_data, sheet_name="操作变量", header=[0, 1], skiprows=[0])
samples = pd.read_excel(io=sample_data, header=[0, 1], skiprows=[0])

# 查看 原始数据情况
# raws 
# 查看 样本数据情况
# samples

### 问题一：

数据处理：请参考近4年的工业数据(见附件一“325个数据样本数据.xlsx”)的预处理结果，依“样本确定方法”（附件二）对285号和313号数据样本进行预处理（原始数据见附件三“285号和313号样本原始数据.xlsx”）并将处理后的数据分别加入到附件一中相应的样本号中，供下面研究使用。

即 数据预处理 ====> 补全数据&修正数据

解题思路：

+ 认真阅读《附件二：样本确定方法.docx》
+ 补全数据：
    - 即需要先确认 样本数据集 $\displaystyle{\mathcal{X}}_{s}$ 中 285号 $x_{s}^{(285)}$ 和 313号 $x_{s}^{(313)}$ 存在属性值缺失的属性集合 $\displaystyle{\mathcal{D}}_{loss}$
    - 将缺失的属性集合 $\displaystyle{\mathcal{D}}_{loss}$ 中的缺失的属性值从 原始数据集 $\displaystyle{\mathcal{X}}_{r}$ 中以求取均值 mean($\displaystyle{\mathcal{V}_{r}}$) 的方式确定
+ 修正数据：
    - 即需要先确认 原始数据集 $\displaystyle{\mathcal{X}}_{r}$ 中 285号 $x_{s}^{(285)}$ 和 313号 $x_{s}^{(313)}$ 样本中各属性的属性值缺失情况，对于缺失情况较大的属性则不对 样本数据集 $\displaystyle{\mathcal{X}}_{s}$ 中的 属性值 $\displaystyle{\mathcal{D}}$ 进行修正
    - 将 285号 $x_{s}^{(285)}$ 和 313号 $x_{s}^{(313)}$ 样本中需要修正的属性采用 变异系数 std($\displaystyle{\mathcal{V}_{r}}$)/mean($\displaystyle{\mathcal{V}_{r}}$) 来判断数据的离散程度，然后依据 正态分布假设检验 $norm(d \sim (mean(\displaystyle{\mathcal{V}_{r}})，std(\displaystyle{\mathcal{V}_{r}})))$，然后使用 依达拉原则 剔除一些不在范围内的样本，然后进行求取均值的操作

代码逻辑：

+ 补全数据：
    - 首先从 样本数据集 $\displaystyle{\mathcal{X}}_{s}$ 中确定样本 285号 $x_{s}^{(285)}$ 和 313号 $x_{s}^{(313)}$ 缺失的属性 $\displaystyle{\mathcal{D}}_{loss}$
    - 然后对缺失的属性 $d_{loss} \in \displaystyle{\mathcal{D}}_{loss}$ 对应到 原始数据集 $\displaystyle{\mathcal{X}}_{r}$ 中，逐一对每个缺失的属性值进行判别
    - 这里使用 $\frac{\displaystyle{\mathbb{I}}(v_{loss}^{(i)} \ne 0|d_{loss}^{(i)}  \in \displaystyle{\mathcal{D}}_{loss})}{\displaystyle{\mathbb{I}}(v_{loss}^{(i)} |d_{loss}^{(i)} \in \displaystyle{\mathcal{D}}_{loss})}$ 对当前 缺失的属性 $d_{loss}$ 在 原始数据集 $\displaystyle{\mathcal{X}}_{r}$ 中数据采集的状况进行评估
        + 当比值 $ \ge \frac{3}{4} $ 时，对当前 缺失的属性的值 $\displaystyle{\mathcal{V}_{d_{loss}}}$ 进行求均值 mean($\displaystyle{\mathcal{V}_{d_{loss}}}$) 处理；
        + 当比值 $ < \frac{3}{4} $ 时，对当前 缺失的属性 $d_{loss}$ 进行直接忽略处理；

+ 修正数据：
    + 对样本 $x_{sample}^{(i)}$ 进行 缺失的属性 $d_{loss}$ 求均值之前需进行异常值处理
        - 根据拉依达准则（3$\sigma$准则）去除异常值，得到 缺失的属性 $d_{loss}$ 对应的过滤后的 原始属性值集合 $\displaystyle{\mathcal{V}}_{filter}^{d_{loss}}$
        - 将处理后的 原始属性值集合 $\displaystyle{\mathcal{V}}_{filter}^{d_{loss}}$ 中的属性值进行求 均值 $\displaystyle{\mathcal{V}}_{filter_{mean}}^{d_{loss}}$ 操作加入到 $\displaystyle{\mathcal{X}}_{s}$ 中对应的 样本数据 $x_{s}^{(i)}$ 对应的 缺失的属性 $d_{loss}$ 中

In [ ]:
# No.285 样本
sample_285 = samples.iloc[284]
sample_285 = sample_285.drop(index=sample_285.index[[x for x in range(1, 16)]], axis=1)
# No.313 样本
sample_313 = samples.iloc[312]
sample_313 = sample_313.drop(index=sample_285.index[[x for x in range(1, 16)]], axis=1)

# 查看 No.285 数据情况
# sample_285
# 查看 No.313 数据情况
# sample_313

In [ ]:
def find_dummy_colmuns(sample):
    dummy_columns = []
    for (code_name, i18n_name) in sample.keys():
        if not sample[(code_name, i18n_name)]:
            # 查看 样本属性值 为 0 的列名
            # print(f"{'='*10}{i18n_name}{'='*10}")
            # print(f"{code_name}: => {sample[(code_name, i18n_name)]}")
            # print(f"{'='*35}")
            dummy_columns.append(code_name)
    return dummy_columns

In [ ]:
sample_285_dummy_columns = find_dummy_colmuns(sample_285)
sample_313_dummy_columns = find_dummy_colmuns(sample_313)

# 查看 No.285 属性值为空的情况
print(f"No.285 属性值为空的列: {sample_285_dummy_columns}")
# 查看 No.313 属性值为空的情况
print(f"No.313 属性值为空的列: {sample_313_dummy_columns}")

In [ ]:
# 拆分原始数据中285号样本的数据
print("拆分出来的285号样本的原始数据")
raw_285 = raws.iloc[:40]
raw_285[sample_285_dummy_columns].head(n=5)

In [ ]:
# 拆分原始数据中313号样本的数据
print("拆分出来的313号样本的原始数据")
raw_313 = raws.iloc[41:]
raw_313[sample_313_dummy_columns].head(n=5)

至此对于原始数据中的 补全数据操作 完成，可惜的是从结果看起来上面所罗列的原始数据并没有对285号和313号样本的缺失的属性值有所帮助

接下来将以原始数据集展开对样本数据集中285号和313号样本的属性值进行修正操作，值得庆幸的是，这一步可以省略上面已经验证的无法补全的一些缺失属性

In [ ]:
# 统计 No.285 样本 中属性中属性值存在空值的属性
raw_285 = raw_285.drop(raw_285[sample_285_dummy_columns], axis=1)
raw_285_nonzeroratio = raw_285.astype(bool).sum(axis=0) / 40
raw_285_dummy_columns = []
for (code_name, i18n_name) in raw_285_nonzeroratio.keys():
    if raw_285_nonzeroratio[(code_name, i18n_name)] != 1.0:
        raw_285_dummy_columns.append(code_name)
print(raw_285_dummy_columns)

In [ ]:
# 依据 拉依达准则 对 No.285 样本进行属性值数据修正
raw_285_filter = raw_285.replace(0, np.NaN)
raw_285_filter_describe = raw_285_filter.describe()
temp_ratio = raw_285_filter_describe.loc['std'] / raw_285_filter_describe.loc['mean']
for (code_name, i18n_name) in temp_ratio.keys():
    if temp_ratio[(code_name, i18n_name)] > 0.3:
        print(f"{'='*10} code_name: {code_name} {'='*10}")
        print(f"样本数据 参照 => {sample_313[(code_name, i18n_name)]}")
        print(f"原始数据 均值 => {raw_285_filter_describe.loc['mean', (code_name, i18n_name)]}")
        print(f"原始数据 方差 => {raw_285_filter_describe.loc['std', (code_name, i18n_name)]}")
        print(f"{'='*40}")
        print(f"变异系数 => {temp_ratio[(code_name, i18n_name)]}")
        norm_test = stats.kstest(raw_285_filter[(code_name, i18n_name)], 'norm', (raw_285_filter_describe.loc['mean', (code_name, i18n_name)], raw_285_filter_describe.loc['std', (code_name, i18n_name)]))
        
        print(f"正态检验 => {norm_test.pvalue}")
        if norm_test.pvalue > 0.05:
            print(f"正态分布检验成功，依据3σ原则进行修正")
            temp_data = raw_285_filter[(code_name, i18n_name)][np.abs(raw_285_filter[(code_name, i18n_name)] - raw_285_filter_describe.loc['mean', (code_name, i18n_name)]) <= 3 * raw_285_filter_describe.loc['std', (code_name, i18n_name)]]
            print(f"修正数据 均值 => {temp_data.mean()}")
            sample_285[(code_name, i18n_name)] = temp_data.mean()
            print(f"样本数据 修正 => {sample_285[(code_name, i18n_name)]}")
        else:
            print(f"正态分布检验失败，不对样本数据进行修正")
        print(f"{'='*40}")
    else:
        sample_285[(code_name, i18n_name)] = raw_285_filter_describe.loc['mean', (code_name, i18n_name)]

In [ ]:
# 统计 No.313 样本 中属性中属性值存在空值的属性
raw_313 = raw_313.drop(raw_313[sample_313_dummy_columns], axis=1)
raw_313_nonzeroratio = raw_313.astype(bool).sum(axis=0) / 40
raw_313_dummy_columns = []
for (code_name, i18n_name) in raw_313_nonzeroratio.keys():
    if raw_313_nonzeroratio[(code_name, i18n_name)] != 1.0:
        raw_313_dummy_columns.append(code_name)
print(raw_313_dummy_columns)

In [ ]:
# 可视化呈现 No.313 样本 中属性中属性值存在空值的属性的分布
import missingno as msno
%matplotlib inline
msno.matrix(raw_313[raw_313_dummy_columns].replace(0, np.nan), labels=True)

In [ ]:
# 对 No.313 样本中 存在属性值缺失的 属性进行统计学描述
dummy_raw_313 = raw_313[raw_313_dummy_columns].replace(0, np.NaN)
dummy_raw_313.describe()

In [ ]:
# 依据 拉依达准则 对 No.313 样本进行属性值数据修正
raw_313_filter = raw_313.drop(raw_313[['S-ZORB.FT_2431.DACA']], axis=1).replace(0, np.NaN)
raw_313_filter_describe = raw_313_filter.describe()
temp_ratio = raw_313_filter_describe.loc['std'] / raw_313_filter_describe.loc['mean']
for (code_name, i18n_name) in temp_ratio.keys():
    if temp_ratio[(code_name, i18n_name)] > 0.3:
        print(f"{'='*10} code_name: {code_name} {'='*10}")
        print(f"样本数据 参照 => {sample_313[(code_name, i18n_name)]}")
        print(f"原始数据 均值 => {raw_313_filter_describe.loc['mean', (code_name, i18n_name)]}")
        print(f"原始数据 方差 => {raw_313_filter_describe.loc['std', (code_name, i18n_name)]}")
        print(f"{'='*40}")
        print(f"变异系数 => {temp_ratio[(code_name, i18n_name)]}")
        norm_test = stats.kstest(raw_313_filter[(code_name, i18n_name)], 'norm', (raw_313_filter_describe.loc['mean', (code_name, i18n_name)], raw_313_filter_describe.loc['std', (code_name, i18n_name)]))
        
        print(f"正态检验 => {norm_test.pvalue}")
        if norm_test.pvalue > 0.05:
            print(f"正态分布检验成功，依据3σ原则进行修正")
            temp_data = raw_313_filter[(code_name, i18n_name)][np.abs(raw_313_filter[(code_name, i18n_name)] - raw_313_filter_describe.loc['mean', (code_name, i18n_name)]) <= 3 * raw_313_filter_describe.loc['std', (code_name, i18n_name)]]
            print(f"修正数据 均值 => {temp_data.mean()}")
            sample_313[(code_name, i18n_name)] = temp_data.mean()
            print(f"样本数据 修正 => {sample_313[(code_name, i18n_name)]}")
        else:
            print(f"正态分布检验失败，不对样本数据进行修正")
        print(f"{'='*40}")
    else:
        sample_313[(code_name, i18n_name)] = raw_313_filter_describe.loc['mean', (code_name, i18n_name)]

### 问题二：

寻找建模主要变量：建立降低辛烷值损失模型涉及包括7个原料性质、2个待生吸附剂性质、2个再生吸附剂性质、2个产品性质等变量以及另外354个操作变量（共计367个变量），工程技术应用中经常使用先降维后建模的方法，这有利于忽略次要因素，发现并分析影响模型的主要变量与因素。因此，请你们根据提供的325个样本数据（见附件一），通过降维的方法从367个操作变量中筛选出建模主要变量，使之尽可能具有代表性、独立性（为了工程应用方便，建议降维后的主要变量在30个以下），并请详细说明建模主要变量的筛选过程及其合理性。（提示：请考虑将原料的辛烷值作为建模变量之一）。

即 特征工程 -- 特征选择&数据降维

解题思路：
+ 特征选择
    - 数据值是否缺失过多：如果一个特征的样本数据缺失较多，则无法统计其对目标的影响，将该变量删除
        - 具体方法：分别统计操作变量的缺失值数量，然后计算其在数据中的比例，超过一定比例的特征可以删除
    - 特征是否发散：如果一个特征不发散，也就是说样本在这个特征上基本上没有差异，这个特征对于样本的区分并没有什么用
        - 具体方法：分别计算操作变量的方差，按方差的大小（需要结合《附件四：354个操作变量信息.xlsx》中的变量阈值进行归一化处理）进行排序（从大到小排），末尾删除一定数量
    - 特征与目标的相关性：这点比较显见，与目标相关性高的特征，应当优选选择，反之则相反
        - 具体方法：利用决策树算法，即采用方差的方法对特征值进行判别，每次选取一个特征值时会

> 在特征选择步骤中如果处理后存在未删除的缺失数据，需要考虑对缺失数据的补充，可以考虑对该数据的分布进行概率分布建模（例如：高斯分布）来确定其值，需要，即在进行数据降维操作之前，需要对数据完整性进行保证

+ 数据降维 (需要注意：“它们的操作变量（控制变量）之间具有高度非线性和相互强耦联的关系”题干中已经明确说明了)
    - 特征之间的相关性：因为操作变量之间是非线性关系，因此无法使用常规的线性降维方式，需要采用流形学习和核化方式对操作变量进行降维
        - 具体方法：利用t-SNE对操作变量进行降维
        

In [ ]:
# 将 No.285 样本和 No.313 样本的修正数据替换到 样本数据 中
samples.iloc[284] = sample_285
samples.iloc[312] = sample_313

samples.iloc[:5, 16:23]

In [ ]:
samples.shape

In [ ]:
samples_nonzeroratio = samples.astype(bool).sum(axis=0) / 325
samples_dummy_columns = []
for (code_name, i18n_name) in samples_nonzeroratio.keys():
    if samples_nonzeroratio[(code_name, i18n_name)] != 1.0:
        print(f"code_name: {code_name} => nonzero_ratio: {samples_nonzeroratio[(code_name, i18n_name)]}")
        samples_dummy_columns.append(code_name)

In [ ]:
# 可视化呈现 样本数据 中属性中属性值存在空值的属性的分布
import missingno as msno
%matplotlib inline
msno.matrix(samples[samples_dummy_columns].replace(0, np.nan), labels=True)

In [ ]:
# 对 缺失值较多属性值 的属性进行丢弃 对 缺失值较少属性值 的属性进行插值
samples_delete_columns = []
samples_insert_columns = []
for (code_name, i18n_name) in samples_nonzeroratio.keys():
    if samples_nonzeroratio[(code_name, i18n_name)] < 0.8:
        # print(f"code_name: {code_name} => nonzero_ratio: {samples_nonzeroratio[(code_name, i18n_name)]}")
        samples_delete_columns.append((code_name, i18n_name))
    elif 0.8 <= samples_nonzeroratio[(code_name, i18n_name)] < 1:
        samples_insert_columns.append((code_name, i18n_name))
# print(samples_insert_columns)
samples = samples.drop(columns=samples_delete_columns, axis=1)
samples = samples.replace(0, np.nan).interpolate()
samples.shape

In [ ]:
# 可视化呈现 样本数据 中属性中属性值插值的情况
import missingno as msno
%matplotlib inline
msno.matrix(samples[samples_insert_columns].replace(0, np.nan), labels=True)

In [ ]:
# 提取 操作属性特征 同时 计算 每个属性与目标值的 相关系数，在 阈值 以下的属性将被丢弃
samples_features = samples.drop(samples.columns[[x for x in range(0, 2)]], axis=1)
samples_features = samples_features.drop(samples_features.columns[[x for x in range(7, 14)]], axis=1)
# 相关系数的阈值
dist_standard1 = 0.2  # 需要调整的参数
samples_delete_columns = []
for (code_name, i18n_name) in samples_features.keys():
    RON_feature_corr = samples[('产品性质', '辛烷值RON')].corr(samples_features[(code_name, i18n_name)])
    corr_dist = np.linalg.norm(RON_feature_corr - 0)
    if corr_dist < dist_standard1:
        samples_delete_columns.append((code_name, i18n_name))
samples_features = samples_features.drop(samples_delete_columns, axis=1)

In [ ]:
samples_features.shape

In [ ]:
y = samples[('产品性质', '辛烷值RON')].to_numpy()
X = samples_features.to_numpy()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold,datasets
# 使用 T-SNE 进行数据降维
# X是特征，不包含target; X_tsne 是已经降维之后的特征
# 指定降维后的维数
re_dimension = 40  # 需要调整的参数
tsne = manifold.TSNE(n_components=re_dimension, init='pca', random_state=501, method="exact")
X_tsne = tsne.fit_transform(X)
print(f"原始样本数据维度 {X.shape[-1]}. t-SNE降维后的数据维度 {X_tsne.shape[-1]}")

In [ ]:
# 根据 新特征属性 与 辛烷值含量RON 的相关系数，筛选 相关系数 大于设定阈值的 新特征属性 作为构建函数目标属性
# 相关系数的阈值
dist_standard2 = 0.1  # 需要调整的参数
tSNE_samples_features = pd.DataFrame(data=X_tsne, columns=[f'feature_{x + 1}' for x in range(re_dimension)])
for _ in tSNE_samples_features.keys():
    RON_feature_corr = samples[('产品性质', '辛烷值RON')].corr(tSNE_samples_features[_])
    corr_dist = np.linalg.norm(RON_feature_corr - 0)
    print(f"{'='*10} 特征名称: {_} {'='*10}")
    print(f"与 辛烷值RON含量 的相关系数: {corr_dist}")
    if corr_dist > dist_standard2:
        print(f"作为评估产品中 辛烷值RON含量 的 特征属性")
    else:
        print(f"不作为评估产品中 辛烷值RON含量 的 特征属性")